# Bedienungsanleitung: Semantische Clustering-Pipeline

Dieses Notebook dient der Steuerung der semantischen Clustering-Pipeline (Team C). Die Pipeline verarbeitet die Daten sessionweise in fünf logischen Schritten.

## 1. Voraussetzungen
Vor der ersten Ausführung müssen die benötigten Python-Pakete (wie `numpy`, `sentence-transformers`, `scipy` und `matplotlib`) installiert werden.

In [ ]:
# Abhängigkeiten installieren
!pip install -r requirements.txt

## 2. Vollständige Pipeline ausführen (End-to-End)

Das zentrale Steuerungsskript lautet `script/build_active_speaker_segments_semantic.py`. 
Die 5 Phasen der Pipeline sind:
1. Textextraktion (`extracted_texts.json`)
2. Embeddings (`embeddings.npy`)
3. Distanzmatrix (`distance_matrix.npy`)
4. Semantic Clustering (`speaker_to_cluster.json`)
5. Evaluation (`evaluation.json`)

Mit dem Argument `--run all` (oder z. B. `--run 1,2,3,4,5`) können alle Schritte sequenziell ausgeführt werden.

In [ ]:
# Gesamten Prozess starten
# Pfade ggf. an die lokale Umgebung anpassen
!python script/build_active_speaker_segments_semantic.py \
  --run all \
  --baseline-dev-dir data-bin/baseline/dev \
  --out-dir data-bin/_output/dev/semantic_clustering

## 3. Gezielte Steuerung und Parameter-Anpassung

Die Pipeline nutzt ein intelligentes Caching. Bereits berechnete Schritte (wie die ressourcenintensiven Embeddings) werden nicht neu berechnet, es sei denn, es wird explizit gefordert. 

Um beispielsweise nur das **Clustering (Schritt 4)** und die **Evaluation (Schritt 5)** mit veränderten Parametern auszuführen, können die Argumente entsprechend angepasst werden:

**Wichtige Parameter für das Clustering:**
* `--cluster-linkage`: Methode der Cluster-Bildung (Optionen: `average`, `complete`, `single`. Standard: `average`).
* `--cluster-distance-threshold`: Der Schwellenwert für die semantische Distanz (Standard: `0.76`).
* `--cluster-n-clusters`: Alternativ zum Threshold kann hier eine feste Anzahl an Clustern vorgegeben werden.
* `--eval-write-csv`: Erzeugt zusätzliche CSV-Exporte der Evaluierungsergebnisse.

In [ ]:
# Nur Clustering und Evaluierung mit spezifischen Parametern ausführen
!python script/build_active_speaker_segments_semantic.py \
  --run 4,5 \
  --baseline-dev-dir data-bin/baseline/dev \
  --out-dir data-bin/_output/dev/semantic_clustering \
  --cluster-linkage average \
  --cluster-distance-threshold 0.76 \
  --eval-write-csv

## 4. Hyperparameter-Optimierung (Grid Search)

Für systematische Versuchsreihen steht das Skript `script/grid_search_semantic_v2.py` zur Verfügung. Es führt das Clustering und die Evaluierung über ein definiertes Raster von Parametern (`linkage` und `threshold`) aus und aggregiert die Ergebnisse.

Die Auswertung wird im Unterordner `_gridsearch` als `grid_results.csv` (Zusammenfassung) und `grid_details.csv` (Details pro Session) gespeichert.

In [ ]:
# Grid Search starten
!python script/grid_search_semantic_v2.py \
  --baseline-root data-bin/baseline/dev \
  --output-root data-bin/_output/dev/semantic_clustering \
  --skip-existing